In [54]:
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

import sys
sys.dont_write_bytecode = True

import numpy as np, cv2
from lxml import etree
from PIL import Image, ImageDraw
import os, json
from collections import OrderedDict
import matplotlib.pyplot as plt
from random import shuffle

In [17]:
clevr_colors = ["red", "blue", "green", "brown", "purple", "cyan", "yellow"]

color_mean_dict = {"red":[140, 64, 61], "yellow":[135, 127, 52],"brown":[112, 84, 46],
                   "cyan":[54, 126, 118], "green":[50, 96, 42],"blue":[50, 70, 120],
                   "purple":[95, 52, 113]}
color_thres_dict = {"red":5, "yellow":5, "brown":5,
                   "cyan":5, "green":5, "blue":5,
                    "purple":5}

In [18]:
def color_range(color_string):
    [red, green, blue] = color_mean_dict[color_string]
    thres = color_thres_dict[color_string]
    
    color = np.uint8([[[blue, green, red]]])
    hsv_color = cv2.cvtColor(color, cv2.COLOR_BGR2HSV)
    hue = hsv_color[0][0][0]
    
    low = max(0, hue - thres)
    high = min(360, hue + thres)
    lower_range = np.array([low, 40, 40], dtype=np.uint8) #very low range for dealing with metallics
    upper_range = np.array([high, 255, 255], dtype=np.uint8)
    
    return (lower_range, upper_range)

# -----------------------------------------------------------------------

color_filter_dict = {x: color_range(x) for x in clevr_colors}

In [19]:
def imshow(img):
    plt.imshow(img[...,::-1])
    plt.pause(0.04)
    return

In [20]:
img_dir = "images/train"
scene_dir = "scenes/train"
img_names = sorted(os.listdir(img_dir))


In [21]:
def load_json(json_path):
    with open(json_path, 'r') as f:
        scene = json.load(f, object_pairs_hook=OrderedDict)
    return scene

In [22]:
def get_bb(img_name, color, pixel_coords, default_size=20, padding=5):
    img = cv2.imread(os.path.join(img_dir, img_name))
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    (low, high) = color_filter_dict[color]
    mask = cv2.inRange(hsv_img, low, high)
    mask = cv2.medianBlur(mask,11)

    (_, cnts, hierarchy) = cv2.findContours(mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

    #     ------Fill all holes---------------
    for i in range(len(cnts)):
        cv2.drawContours(mask, cnts, i, (255,255,255), cv2.FILLED)

    # ---------------------------------------
    (_, cnts, hierarchy) = cv2.findContours(mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

    if(len(cnts) == 0):
        xmin = pixel_coords[0] - default_size; ymin = pixel_coords[1] - default_size
        xmax = pixel_coords[0] + default_size; ymax = pixel_coords[1] + default_size
    elif(cv2.contourArea(cnts[0]) < 20):
        xmin = pixel_coords[0] - default_size; ymin = pixel_coords[1] - default_size
        xmax = pixel_coords[0] + default_size; ymax = pixel_coords[1] + default_size
    else:
        rect = cv2.boundingRect(cnts[0])
        xmin,ymin,w,h = rect
        xmax = xmin + w; ymax = ymin + h
    
    xmin = xmin - padding; ymin = ymin - padding
    xmax = xmax + padding; ymax = ymax + padding
    
    height, width, channels = img.shape
    xmin = max(0, xmin); ymin = max(0, ymin)
    xmax = min(width, xmax); ymax = min(height, ymax)
    
#     cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (255,0,0), 2)
#     imshow(img)

    return (xmin, ymin, xmax, ymax)

In [23]:
def get_annotations(img_name):
    scene = load_json(os.path.join(scene_dir, img_name[:-4] + ".json"))
    object_info = scene["objects"][0]
    size = object_info["size"]
    shape = object_info["shape"]
    material = object_info["material"]
    color = object_info["color"]
#     x = object_info["3d_coords"][0]
#     y = object_info["3d_coords"][1]
#     theta = object_info["rotation"]
#     bb is (xmin, ymin, xmax, ymax)
    bb = get_bb(img_name, color, object_info["pixel_coords"])

    dict = {"size": size, "shape": shape, "material": material, "bb":bb}
    return dict
    

In [47]:
def write_xml(img_name):
    dict = get_annotations(img_name)
    
    img = Image.open(os.path.join(img_dir, img_name))
    bg = Image.new("RGB", img.size, (255,255,255))
    bg.paste(img,img)
    
    img_name = img_name[:-4]+".JPEG" #update to jpg
    
    bg.save(r"JPEGImages/"+img_name)
    width, height = img.size
    
    
    root = etree.Element("annotation")
    etree.SubElement(root, "folder").text = "clevr_single"
    etree.SubElement(root, "filename").text = img_name

    source = etree.SubElement(root, "source")
    etree.SubElement(source, "database").text = "CLEVR_SINGLE"

    size = etree.SubElement(root, "size")
    etree.SubElement(size, "width").text = str(width)
    etree.SubElement(size, "height").text = str(height)
    etree.SubElement(size, "depth").text = str(3)

    etree.SubElement(root, "segmented").text = str(0)

    object = etree.SubElement(root, "object")
    etree.SubElement(object, "name").text = str(dict["shape"])
    etree.SubElement(object, "pose").text = "unspecified"
    etree.SubElement(object, "truncated").text = str(0)
    etree.SubElement(object, "difficult").text = str(0)
    bndbox = etree.SubElement(object, "bndbox")
    etree.SubElement(bndbox, "xmin").text = str(dict["bb"][0])
    etree.SubElement(bndbox, "ymin").text = str(dict["bb"][1])
    etree.SubElement(bndbox, "xmax").text = str(dict["bb"][2])
    etree.SubElement(bndbox, "ymax").text = str(dict["bb"][3])

    tree = etree.ElementTree(root)
    tree.write("Annotations/"+img_name[:-5]+".xml",pretty_print=True )
    
    return

In [48]:
img_dir = "images/train"
img_names = sorted(os.listdir(img_dir))


In [49]:
len(img_names)

6000

In [50]:
write_xml(img_names[0])

In [51]:
for i in range(6000):
    print(img_names[i])
    write_xml(img_names[i])


CLEVR_train_000000.png
CLEVR_train_000001.png
CLEVR_train_000002.png
CLEVR_train_000003.png
CLEVR_train_000004.png
CLEVR_train_000005.png
CLEVR_train_000006.png
CLEVR_train_000007.png
CLEVR_train_000008.png
CLEVR_train_000009.png
CLEVR_train_000010.png
CLEVR_train_000011.png
CLEVR_train_000012.png
CLEVR_train_000013.png
CLEVR_train_000014.png
CLEVR_train_000015.png
CLEVR_train_000016.png
CLEVR_train_000017.png
CLEVR_train_000018.png
CLEVR_train_000019.png
CLEVR_train_000020.png
CLEVR_train_000021.png
CLEVR_train_000022.png
CLEVR_train_000023.png
CLEVR_train_000024.png
CLEVR_train_000025.png
CLEVR_train_000026.png
CLEVR_train_000027.png
CLEVR_train_000028.png
CLEVR_train_000029.png
CLEVR_train_000030.png
CLEVR_train_000031.png
CLEVR_train_000032.png
CLEVR_train_000033.png
CLEVR_train_000034.png
CLEVR_train_000035.png
CLEVR_train_000036.png
CLEVR_train_000037.png
CLEVR_train_000038.png
CLEVR_train_000039.png
CLEVR_train_000040.png
CLEVR_train_000041.png
CLEVR_train_000042.png
CLEVR_train

CLEVR_train_000361.png
CLEVR_train_000362.png
CLEVR_train_000363.png
CLEVR_train_000364.png
CLEVR_train_000365.png
CLEVR_train_000366.png
CLEVR_train_000367.png
CLEVR_train_000368.png
CLEVR_train_000369.png
CLEVR_train_000370.png
CLEVR_train_000371.png
CLEVR_train_000372.png
CLEVR_train_000373.png
CLEVR_train_000374.png
CLEVR_train_000375.png
CLEVR_train_000376.png
CLEVR_train_000377.png
CLEVR_train_000378.png
CLEVR_train_000379.png
CLEVR_train_000380.png
CLEVR_train_000381.png
CLEVR_train_000382.png
CLEVR_train_000383.png
CLEVR_train_000384.png
CLEVR_train_000385.png
CLEVR_train_000386.png
CLEVR_train_000387.png
CLEVR_train_000388.png
CLEVR_train_000389.png
CLEVR_train_000390.png
CLEVR_train_000391.png
CLEVR_train_000392.png
CLEVR_train_000393.png
CLEVR_train_000394.png
CLEVR_train_000395.png
CLEVR_train_000396.png
CLEVR_train_000397.png
CLEVR_train_000398.png
CLEVR_train_000399.png
CLEVR_train_000400.png
CLEVR_train_000401.png
CLEVR_train_000402.png
CLEVR_train_000403.png
CLEVR_train

CLEVR_train_000721.png
CLEVR_train_000722.png
CLEVR_train_000723.png
CLEVR_train_000724.png
CLEVR_train_000725.png
CLEVR_train_000726.png
CLEVR_train_000727.png
CLEVR_train_000728.png
CLEVR_train_000729.png
CLEVR_train_000730.png
CLEVR_train_000731.png
CLEVR_train_000732.png
CLEVR_train_000733.png
CLEVR_train_000734.png
CLEVR_train_000735.png
CLEVR_train_000736.png
CLEVR_train_000737.png
CLEVR_train_000738.png
CLEVR_train_000739.png
CLEVR_train_000740.png
CLEVR_train_000741.png
CLEVR_train_000742.png
CLEVR_train_000743.png
CLEVR_train_000744.png
CLEVR_train_000745.png
CLEVR_train_000746.png
CLEVR_train_000747.png
CLEVR_train_000748.png
CLEVR_train_000749.png
CLEVR_train_000750.png
CLEVR_train_000751.png
CLEVR_train_000752.png
CLEVR_train_000753.png
CLEVR_train_000754.png
CLEVR_train_000755.png
CLEVR_train_000756.png
CLEVR_train_000757.png
CLEVR_train_000758.png
CLEVR_train_000759.png
CLEVR_train_000760.png
CLEVR_train_000761.png
CLEVR_train_000762.png
CLEVR_train_000763.png
CLEVR_train

CLEVR_train_001082.png
CLEVR_train_001083.png
CLEVR_train_001084.png
CLEVR_train_001085.png
CLEVR_train_001086.png
CLEVR_train_001087.png
CLEVR_train_001088.png
CLEVR_train_001089.png
CLEVR_train_001090.png
CLEVR_train_001091.png
CLEVR_train_001092.png
CLEVR_train_001093.png
CLEVR_train_001094.png
CLEVR_train_001095.png
CLEVR_train_001096.png
CLEVR_train_001097.png
CLEVR_train_001098.png
CLEVR_train_001099.png
CLEVR_train_001100.png
CLEVR_train_001101.png
CLEVR_train_001102.png
CLEVR_train_001103.png
CLEVR_train_001104.png
CLEVR_train_001105.png
CLEVR_train_001106.png
CLEVR_train_001107.png
CLEVR_train_001108.png
CLEVR_train_001109.png
CLEVR_train_001110.png
CLEVR_train_001111.png
CLEVR_train_001112.png
CLEVR_train_001113.png
CLEVR_train_001114.png
CLEVR_train_001115.png
CLEVR_train_001116.png
CLEVR_train_001117.png
CLEVR_train_001118.png
CLEVR_train_001119.png
CLEVR_train_001120.png
CLEVR_train_001121.png
CLEVR_train_001122.png
CLEVR_train_001123.png
CLEVR_train_001124.png
CLEVR_train

CLEVR_train_001445.png
CLEVR_train_001446.png
CLEVR_train_001447.png
CLEVR_train_001448.png
CLEVR_train_001449.png
CLEVR_train_001450.png
CLEVR_train_001451.png
CLEVR_train_001452.png
CLEVR_train_001453.png
CLEVR_train_001454.png
CLEVR_train_001455.png
CLEVR_train_001456.png
CLEVR_train_001457.png
CLEVR_train_001458.png
CLEVR_train_001459.png
CLEVR_train_001460.png
CLEVR_train_001461.png
CLEVR_train_001462.png
CLEVR_train_001463.png
CLEVR_train_001464.png
CLEVR_train_001465.png
CLEVR_train_001466.png
CLEVR_train_001467.png
CLEVR_train_001468.png
CLEVR_train_001469.png
CLEVR_train_001470.png
CLEVR_train_001471.png
CLEVR_train_001472.png
CLEVR_train_001473.png
CLEVR_train_001474.png
CLEVR_train_001475.png
CLEVR_train_001476.png
CLEVR_train_001477.png
CLEVR_train_001478.png
CLEVR_train_001479.png
CLEVR_train_001480.png
CLEVR_train_001481.png
CLEVR_train_001482.png
CLEVR_train_001483.png
CLEVR_train_001484.png
CLEVR_train_001485.png
CLEVR_train_001486.png
CLEVR_train_001487.png
CLEVR_train

...

CLEVR_train_004341.png
CLEVR_train_004342.png
CLEVR_train_004343.png
CLEVR_train_004344.png
CLEVR_train_004345.png
CLEVR_train_004346.png
CLEVR_train_004347.png
CLEVR_train_004348.png
CLEVR_train_004349.png
CLEVR_train_004350.png
CLEVR_train_004351.png
CLEVR_train_004352.png
CLEVR_train_004353.png
CLEVR_train_004354.png
CLEVR_train_004355.png
CLEVR_train_004356.png
CLEVR_train_004357.png
CLEVR_train_004358.png
CLEVR_train_004359.png
CLEVR_train_004360.png
CLEVR_train_004361.png
CLEVR_train_004362.png
CLEVR_train_004363.png
CLEVR_train_004364.png
CLEVR_train_004365.png
CLEVR_train_004366.png
CLEVR_train_004367.png
CLEVR_train_004368.png
CLEVR_train_004369.png
CLEVR_train_004370.png
CLEVR_train_004371.png
CLEVR_train_004372.png
CLEVR_train_004373.png
CLEVR_train_004374.png
CLEVR_train_004375.png
CLEVR_train_004376.png
CLEVR_train_004377.png
CLEVR_train_004378.png
CLEVR_train_004379.png
CLEVR_train_004380.png
CLEVR_train_004381.png
CLEVR_train_004382.png
CLEVR_train_004383.png
CLEVR_train

CLEVR_train_004706.png
CLEVR_train_004707.png
CLEVR_train_004708.png
CLEVR_train_004709.png
CLEVR_train_004710.png
CLEVR_train_004711.png
CLEVR_train_004712.png
CLEVR_train_004713.png
CLEVR_train_004714.png
CLEVR_train_004715.png
CLEVR_train_004716.png
CLEVR_train_004717.png
CLEVR_train_004718.png
CLEVR_train_004719.png
CLEVR_train_004720.png
CLEVR_train_004721.png
CLEVR_train_004722.png
CLEVR_train_004723.png
CLEVR_train_004724.png
CLEVR_train_004725.png
CLEVR_train_004726.png
CLEVR_train_004727.png
CLEVR_train_004728.png
CLEVR_train_004729.png
CLEVR_train_004730.png
CLEVR_train_004731.png
CLEVR_train_004732.png
CLEVR_train_004733.png
CLEVR_train_004734.png
CLEVR_train_004735.png
CLEVR_train_004736.png
CLEVR_train_004737.png
CLEVR_train_004738.png
CLEVR_train_004739.png
CLEVR_train_004740.png
CLEVR_train_004741.png
CLEVR_train_004742.png
CLEVR_train_004743.png
CLEVR_train_004744.png
CLEVR_train_004745.png
CLEVR_train_004746.png
CLEVR_train_004747.png
CLEVR_train_004748.png
CLEVR_train

CLEVR_train_005068.png
CLEVR_train_005069.png
CLEVR_train_005070.png
CLEVR_train_005071.png
CLEVR_train_005072.png
CLEVR_train_005073.png
CLEVR_train_005074.png
CLEVR_train_005075.png
CLEVR_train_005076.png
CLEVR_train_005077.png
CLEVR_train_005078.png
CLEVR_train_005079.png
CLEVR_train_005080.png
CLEVR_train_005081.png
CLEVR_train_005082.png
CLEVR_train_005083.png
CLEVR_train_005084.png
CLEVR_train_005085.png
CLEVR_train_005086.png
CLEVR_train_005087.png
CLEVR_train_005088.png
CLEVR_train_005089.png
CLEVR_train_005090.png
CLEVR_train_005091.png
CLEVR_train_005092.png
CLEVR_train_005093.png
CLEVR_train_005094.png
CLEVR_train_005095.png
CLEVR_train_005096.png
CLEVR_train_005097.png
CLEVR_train_005098.png
CLEVR_train_005099.png
CLEVR_train_005100.png
CLEVR_train_005101.png
CLEVR_train_005102.png
CLEVR_train_005103.png
CLEVR_train_005104.png
CLEVR_train_005105.png
CLEVR_train_005106.png
CLEVR_train_005107.png
CLEVR_train_005108.png
CLEVR_train_005109.png
CLEVR_train_005110.png
CLEVR_train

CLEVR_train_005432.png
CLEVR_train_005433.png
CLEVR_train_005434.png
CLEVR_train_005435.png
CLEVR_train_005436.png
CLEVR_train_005437.png
CLEVR_train_005438.png
CLEVR_train_005439.png
CLEVR_train_005440.png
CLEVR_train_005441.png
CLEVR_train_005442.png
CLEVR_train_005443.png
CLEVR_train_005444.png
CLEVR_train_005445.png
CLEVR_train_005446.png
CLEVR_train_005447.png
CLEVR_train_005448.png
CLEVR_train_005449.png
CLEVR_train_005450.png
CLEVR_train_005451.png
CLEVR_train_005452.png
CLEVR_train_005453.png
CLEVR_train_005454.png
CLEVR_train_005455.png
CLEVR_train_005456.png
CLEVR_train_005457.png
CLEVR_train_005458.png
CLEVR_train_005459.png
CLEVR_train_005460.png
CLEVR_train_005461.png
CLEVR_train_005462.png
CLEVR_train_005463.png
CLEVR_train_005464.png
CLEVR_train_005465.png
CLEVR_train_005466.png
CLEVR_train_005467.png
CLEVR_train_005468.png
CLEVR_train_005469.png
CLEVR_train_005470.png
CLEVR_train_005471.png
CLEVR_train_005472.png
CLEVR_train_005473.png
CLEVR_train_005474.png
CLEVR_train

CLEVR_train_005795.png
CLEVR_train_005796.png
CLEVR_train_005797.png
CLEVR_train_005798.png
CLEVR_train_005799.png
CLEVR_train_005800.png
CLEVR_train_005801.png
CLEVR_train_005802.png
CLEVR_train_005803.png
CLEVR_train_005804.png
CLEVR_train_005805.png
CLEVR_train_005806.png
CLEVR_train_005807.png
CLEVR_train_005808.png
CLEVR_train_005809.png
CLEVR_train_005810.png
CLEVR_train_005811.png
CLEVR_train_005812.png
CLEVR_train_005813.png
CLEVR_train_005814.png
CLEVR_train_005815.png
CLEVR_train_005816.png
CLEVR_train_005817.png
CLEVR_train_005818.png
CLEVR_train_005819.png
CLEVR_train_005820.png
CLEVR_train_005821.png
CLEVR_train_005822.png
CLEVR_train_005823.png
CLEVR_train_005824.png
CLEVR_train_005825.png
CLEVR_train_005826.png
CLEVR_train_005827.png
CLEVR_train_005828.png
CLEVR_train_005829.png
CLEVR_train_005830.png
CLEVR_train_005831.png
CLEVR_train_005832.png
CLEVR_train_005833.png
CLEVR_train_005834.png
CLEVR_train_005835.png
CLEVR_train_005836.png
CLEVR_train_005837.png
CLEVR_train

In [58]:
shuffle(img_names)
train_val_split = 0.8
end_idx = round(len(img_names) * train_val_split)
train_img_names = img_names[:end_idx]
val_img_names = img_names[end_idx:]

In [59]:
len(train_img_names)

4800

In [60]:
len(val_img_names)

1200